## Question 1 and 2
Copy A* algorithm from pathfinding.ipynb

## Question 3

In [97]:
class BoardConfiguration:
    def __init__(self, config: list[int]):
        self.config = config

    def is_solved(self) -> bool:
        '''Verify if the board is solved'''
        return all([i + 1 == self.config[i] for i in range(8)])
    
    def neighbors(self) -> list:
        '''Return a list of the configurations 1 move away'''
        # may lord forgive me for writing this method
        blank_index = self.config.index(None)
        result = []

        for offset in [-1, 1, -3, 3]:
            swap_index = blank_index + offset

            if (abs(offset) == 1 and swap_index // 3 != blank_index // 3):
                continue
            
            if swap_index in range(9):
                tmp = self.config.copy()
                tmp[swap_index], tmp[blank_index] = None, tmp[swap_index]
                result.append(tmp)

        return [BoardConfiguration(i) for i in result] 
    
    def __hash__(self) -> int:
        '''Return the hash value of the config
        Hash value := index of the blank space
        '''
        return self.config.index(None)
    
    def __lt__(self, other) -> bool:
        '''Compare on some arbitrary metric
        Needed for hashing
        '''
        return self.config.index(None) < other.config.index(None)
    
    def heuristic(self) -> int:
        '''Return the heuristic value of the board configuration'''
        h = 0
        for i in range(9):
            if self.config[i] != None and self.config[i] != i + 1:
                h += 1
        return h

    def display(self) -> None:
        '''Print the board'''
        print("-" * 5)
        for i in range(3):
            for j in range(3):
                value = self.config[i * 3 + j]
                print(value if value else " ", end = " ")
            print()
        print("-" * 5)

In [98]:
def backtrack(end, prev) -> list[BoardConfiguration]:
    '''Return backtracked path from start to end using prev dictionary'''
    path = []
    current = end
    while prev[current]:
        path.append(current)
        current = prev[current]
    return path[::-1]


In [99]:
from heapq import heappush, heappop
from collections import defaultdict

def a_star(start) -> list[BoardConfiguration]:
    '''Return a list of BoardConfigurations from start to solved'''
    dist = defaultdict(lambda: float("inf"))  # initial distance estimates
    dist[start] = 0

    prev = defaultdict(lambda: None)
    prev[start] = None

    frontier = [(0, start)] # "frontier" to be explored 
    closed = set()

    while frontier:
        priority_current, current = heappop(frontier)
        
        if current.is_solved():
            break

        if current in closed: 
            continue
        closed.add(current)

        for neighbor in current.neighbors():
            alt_dist = dist[current] + 1 
            
            if alt_dist < dist[neighbor]:
                dist[neighbor] = alt_dist
                prev[neighbor] = current

                priority = alt_dist + neighbor.heuristic()
                heappush(frontier, (priority, neighbor)) 

    return backtrack(current, prev)

In [100]:
B = BoardConfiguration([1, 2, 3, None, 4, 6, 7, 5, 8])
path = a_star(B)
for b in path:
    b.display()

-----
1 2 3 
4   6 
7 5 8 
-----
-----
1 2 3 
4 5 6 
7   8 
-----
-----
1 2 3 
4 5 6 
7 8   
-----
